##### Some description how it works:
Every piece is tracked in `info` dictionary.
It is not allowed for a piece to move to it's visited places.
Depth of search of optimal move is limited at 5, because algorithm would take very long to calculate the MOST optimal move, and we need to do it on every step, which is not acceptable on interactive game, so I decided to limit the depth of search. Limit picked up by experimenting. Still there will be some delay for bot's response.

In [1]:
import numpy as np

'''
Two players. Empty cell denoted by '-'.
'''
player, opponent = 'x', 'o'


'''
Checking if the coordinates are on board.
'''
def onBoard(tpl):
    row = tpl[0]
    col = tpl[1]
    if row >= 0 and row <=3 and col >= 0 and col <= 3:
        return True
    else:
        return False

    
'''
Checking if the cell on board is empty.
'''
def isEmpty(tpl, b):
    row = tpl[0]
    col = tpl[1]
    if b[row][col] == '-':
        return True
    else:
        return False

    
'''
Returning possible moves for a given piece coordinates.
'''
def possibleMoves(tpl, b, info):
    row = tpl[0]
    col = tpl[1]
    arr = []
    if b[row][col] == player or b[row][col] == opponent:
        candidateMoves = [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]
        for candidateMove in candidateMoves:
            if onBoard(candidateMove) and isEmpty(candidateMove, b) and (candidateMove not in info[(row, col)]):
                arr.append(candidateMove)
    return arr


'''
Looking for a winning combination.
'''
def allThreeEqual(arr):
    if arr[0] == arr[1] and arr[1] == arr[2]:
        return True
    else:
        return False
    
    
'''
Just printing the board.
'''
def printBoard(b):
    print("\n")
    for i in range(4):
        print(i, "|", b[i][0], "", b[i][1], "", b[i][2], "", b[i][3])
    print("  -------------")
    print("    0  1  2  3")


'''
Initializing INFO dictionary from board arrangement.
'''
def getInfo(b):
    d = dict()
    for i in range(4):
        for j in range(4):
            if b[i][j] != "-":
                d[(i, j)] = []
    return d


'''
Return the score of the board.
'''
def evaluate(b) :

    for row in range(4) :
        if allThreeEqual(b[row][0:3]) or allThreeEqual(b[row][1:4]):
            if (b[row][1] == player) :
                return 10
            elif (b[row][1] == opponent) :
                return -10

    b_T = np.array(b).T
    for col in range(4) :
        if allThreeEqual(b_T[col][0:3]) or allThreeEqual(b_T[col][1:4]):
            if (b_T[col][1] == player) :
                return 10
            elif (b_T[col][1] == opponent) :
                return -10

    for row in range(2):
        for col in range(2):
            if allThreeEqual([b[row][col], b[row+1][col+1], b[row+2][col+2]]):
                if (b[row][col] == player):
                    return 10
                elif (b[row][col] == opponent):
                    return -10
                
    for row in range(2, 4):
        for col in range(2):
            if allThreeEqual([b[row][col], b[row-1][col+1], b[row-2][col+2]]):
                if (b[row][col] == player):
                    return 10
                elif (b[row][col] == opponent):
                    return -10

    return 0


depth_level = 5


'''
Recursive minimax algorithm.
'''
def minimax(board, depth, isMax, info) :
    
    score = evaluate(board)

    if (score == 10) :
        return score

    if (score == -10) :
        return score

    if (isMax) :	
        best = -1000
        
        if depth > depth_level:
            return best

        for i in range(4):
            for j in range(4):
                if board[i][j] == player:
                    possible_moves = possibleMoves((i, j), board, info)

                    for move in possible_moves:
#                         print("from ROW:", i, " COL:", j, " to ROW:", move[0], " COL:", move[1], " depth:", depth)
            
                        temp = info[(i, j)]
                        info.pop((i, j))
                        info[(move[0], move[1])] = temp + [(i, j)]
                        
                        board[i][j] = '-'
                        board[move[0]][move[1]] = player

#                         printBoard(board)
                        best = max( best, minimax(board,
                                                depth + 1,
                                                not isMax, info) )
                        
                        info.pop((move[0], move[1]))
                        info[(i, j)] = temp

                        board[move[0]][move[1]] = '-'
                        board[i][j] = player
                    
        return best

    else :
        best = 1000
        
        if depth > depth_level:
            return best

        for i in range(4):
            for j in range(4):
                if board[i][j] == opponent:
                    possible_moves = possibleMoves((i, j), board, info)

                    for move in possible_moves:
#                         print("from ROW:", i, " COL:", j, " to ROW:", move[0], " COL:", move[1], " depth:", depth)
                        
                        temp = info[(i, j)]
                        info.pop((i, j))
                        info[(move[0], move[1])] = temp + [(i, j)]
    
                        board[i][j] = '-'
                        board[move[0]][move[1]] = opponent

#                         printBoard(board)
                        best = min( best, minimax(board,
                                                depth + 1,
                                                not isMax, info) )

                        info.pop((move[0], move[1]))
                        info[(i, j)] = temp
                    
                        board[move[0]][move[1]] = '-'
                        board[i][j] = opponent
                    
        return best

    
'''
Finding the best move for current board arrangement.
'''
def findBestMove(board) :
    bestVal = -1000
    bestMove = [(-1, -1), (-2, -2)]
    info = getInfo(board)
    
    for i in range(4):
        for j in range(4):
            if board[i][j] == opponent:
                possible_moves = possibleMoves((i, j), board, info)

                for move in possible_moves:
                    
                    temp = info[(i, j)]
                    info.pop((i, j))
                    info[(move[0], move[1])] = temp + [(i, j)]
                    
                    board[i][j] = '-'
                    board[move[0]][move[1]] = opponent

#                     printBoard(board)
                    moveVal = minimax(board, 0, True, info)

                    info.pop((move[0], move[1]))
                    info[(i, j)] = temp
    
                    board[move[0]][move[1]] = '-'
                    board[i][j] = opponent

                    if (moveVal > bestVal):
                        bestMove = [(i, j), (move[0], move[1])]
                        bestVal = moveVal
                    
    return bestMove

In [2]:
print("Let's start Taktikl game!")

board = [
    [ 'x', 'o', 'x', 'o' ],
    [ '-', '-', '-', '-' ],
    [ '-', '-', '-', '-' ],
    [ 'o', 'x', 'o', 'x' ]
]

print("Here is the board:")
printBoard(board)
print("\nYou are STARTING! You will play with Xs.\n")

while True:
    
    print("Which one you want to move? Format is following:\nROW,COLUMN")
    i = input()
    
    if i == 'q':
        print("Bye!")
        break
        
    l = i.split(',')
    row1, col1 = int(l[0]), int(l[1])

    print("Where you want to move it? Format is following:\nROW,COLUMN")
    i = input()
    
    if i == 'q':
        print("Bye!")
        break
    
    l = i.split(',')
    row2, col2 = int(l[0]), int(l[1])

    board[row1][col1] = '-'
    board[row2][col2] = 'x'
    
    printBoard(board)
    
    print("Bot's turn:")
    print("Warning: it may take SOME TIME for algorithm to search for optimal move. Please don't close.")
    
    bestMove = findBestMove(board)
    board[ bestMove[0][0] ][ bestMove[0][1] ] = '-'
    board[ bestMove[1][0] ][ bestMove[1][1] ] = 'o'
    
    printBoard(board)

Let's start Taktikl game!
Here is the board:


0 | x  o  x  o
1 | -  -  -  -
2 | -  -  -  -
3 | o  x  o  x
  -------------
    0  1  2  3

You are STARTING! You will play with Xs.

Which one you want to move? Format is following:
ROW,COLUMN
0,0
Where you want to move it? Format is following:
ROW,COLUMN
1,0


0 | -  o  x  o
1 | x  -  -  -
2 | -  -  -  -
3 | o  x  o  x
  -------------
    0  1  2  3
Bot's turn:


0 | o  -  x  o
1 | x  -  -  -
2 | -  -  -  -
3 | o  x  o  x
  -------------
    0  1  2  3
Which one you want to move? Format is following:
ROW,COLUMN
q
Bye!
